In [1]:
from src.dl.loader import BlockRegressDataset
import numpy as np
from torch import nn
from torch.utils.data import DataLoader, sampler
from torch.utils.tensorboard import SummaryWriter
import torch
from sklearn.model_selection import KFold
log = False

In [7]:
log = True
writer = SummaryWriter('./log')

In [2]:
from src.utils.data import getPandas, writePandas
data = getPandas('data')
test = data.sample(frac=0.2, random_state=10)
data = data.drop(test.index)
validate = data.sample(frac=0.1, random_state=10)
train = data.drop(validate.index)
test = test.reset_index(drop=True)
train = train.reset_index(drop=True)
validate = validate.reset_index(drop=True)
writePandas('data_test', test)
writePandas('data_train', train)
writePandas('data_validate', validate)

In [3]:
from src.dl.loader import BlockRegressDataset
from src.dl.resnet import RegressResNet3d
from sklearn.model_selection import KFold
import collections
bounds = [0.3]
train_set = BlockRegressDataset('train')
val_set = BlockRegressDataset('validate')
test_set = BlockRegressDataset('test')
fold_num = 5
kf = KFold(n_splits=fold_num, shuffle=True, random_state=10)
best_models = np.empty(dtype=collections.OrderedDict, shape=fold_num)

In [4]:
net = RegressResNet3d().cuda()
loss_fn = nn.SmoothL1Loss().cuda()
#lr = 1e-2
#optim = torch.optim.Adam(net.parameters(), lr=lr)
lr = 1e-2
optim = torch.optim.SGD([{'params': net.fc.weight, 'lr': 1e-2}], lr=lr)
epoch = 100
train_loader = DataLoader(train_set, batch_size=64)
val_loader = DataLoader(val_set, batch_size=64)
for i in range(epoch):
    total_loss = 0
    predy = np.array([])
    y = np.array([])
    net.train()
    for step, [img, labels, score] in enumerate(train_loader):
        img = img.cuda()
        labels = labels.cuda()
        score = score.cuda()
        output = net(img, labels).squeeze(-1)
        loss = loss_fn(output, score)

        optim.zero_grad()
        loss.backward()
        optim.step()
        #scheduler.step()
        
    net.eval()
    with torch.no_grad():
        total = 0
        correct = 0
        for step, [img, labels, score] in enumerate(val_loader):
            img = img.cuda()
            labels = labels.cuda()
            score = score.cuda()
            output = net(img, labels).squeeze(-1)
            loss = loss_fn(output, score)

            predy = np.concatenate((predy, output.cpu().detach().numpy()), axis=0)
            y = np.concatenate((y, score.cpu().detach().numpy()), axis=0)
            total_loss += loss.item()

    nmse = np.sum((y - predy) ** 2) / np.sum(y ** 2)
    mse = np.sum((y - predy) ** 2) / len(val_set)
    rmse = np.sqrt(mse)
    mae = np.sum(np.abs(y - predy)) / len(val_set)
    meany = np.mean(y)
    rsquared = 1 - np.sum((predy - y) ** 2) / np.sum((y - meany) ** 2)
    print('epoch {}, loss {}, nmse {}, rmse {}, mae {}, rsquared {}'.format(i, total_loss, nmse, rmse, mae, rsquared))
    if log:
        writer.add_scalar('Loss', total_loss, i)
        writer.add_scalar('NMSE', nmse, i)
        writer.add_scalar('RMSE', rmse, i)
        writer.add_scalar('MAE', mae, i)
        writer.add_scalar('rsquared', rsquared, i)

epoch 0, loss 0.8890895610878442, nmse 0.4053392229372104, rmse 0.2059038293337789, mae 0.16096915202992548, rsquared -0.10505371485602777
epoch 1, loss 0.9243714587538152, nmse 0.42143348351351156, rmse 0.20995181203332292, mae 0.16251112843978868, rsquared -0.14893059977436063
epoch 2, loss 0.9562745760402215, nmse 0.43598496659768965, rmse 0.21354571704674985, mae 0.16472418246484674, rsquared -0.18860149646753976


KeyboardInterrupt: 

In [5]:
test_loader = DataLoader(test_set, batch_size=64)
net.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for step, [img, labels, score] in enumerate(test_loader):
        img = img.cuda()
        labels = labels.cuda()
        score = score.cuda()
        #score = torch.Tensor(np.random.randint(0, 3, score.size())).type(torch.LongTensor).cuda()
        output = net(img, labels).squeeze(-1)
        loss = loss_fn(output, score)
        
        predy = np.concatenate((predy, output.cpu().detach().numpy()), axis=0)
        y = np.concatenate((y, score.cpu().detach().numpy()), axis=0)
        total_loss += loss.item()
nmse = np.sum((y - predy) ** 2) / np.sum(y ** 2)
mse = np.sum((y - predy) ** 2) / len(test_set)
rmse = np.sqrt(mse)
mae = np.sum(np.abs(y - predy)) / len(test_set)
meany = np.mean(y)
rsquared = 1 - np.sum((predy - y) ** 2) / np.sum((y - meany) ** 2)
print('epoch {}, loss {}, nmse {}, rmse {}, mae {}, rsquared {}'.format(i, total_loss, nmse, rmse, mae, rsquared))

epoch 5, loss 1.5681449702312622, nmse 0.188543016723713, rmse 0.17454700834776615, mae 0.14652274216612954, rsquared 0.1267262404420854


In [ ]:
loss_fn = nn.SmoothL1Loss().cuda()
lr = 1e-3
optim = torch.optim.SGD([
    {'params': net.fc.weight, 'lr': 1e-2}
    ], lr=lr, momentum=0.3)
#optim = torch.optim.SGD(net.parameters(), lr=1e-1, momentum=0.3)
#scheduler = torch.optim.lr_scheduler.ExponentialLR(optim, gamma=0.99)
#optim = torch.optim.Adam(net.parameters(), lr=lr, betas=[0.3, 0.1])
epoch = 100
dataset = BlockRegressDataset('train')
kf = KFold(n_splits=5, shuffle=True, random_state=10)
for fold, (train_idx, val_idx) in enumerate(kf.split(np.arange(len(dataset)))):
    train_sampler = sampler.SubsetRandomSampler(train_idx)
    val_sampler = sampler.SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size=64, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=64, sampler=val_sampler)
    for i in range(epoch):
        total_loss = 0
        predy = np.array([])
        y = np.array([])
        net.train()
        for step, [img, labels, score] in enumerate(train_loader):
            img = img.cuda()
            labels = labels.cuda()
            score = score.cuda()
            output = net(img, labels).squeeze(-1)
            loss = loss_fn(output.float(), score.float())

            optim.zero_grad()
            loss.backward()
            optim.step()
            #scheduler.step()
        
        net.eval()
        with torch.no_grad():
            for step, [img, labels, score] in enumerate(val_loader):
                img = img.cuda()
                labels = labels.cuda()
                score = score.cuda()
                output = net(img, labels).squeeze(-1)
                loss = loss_fn(output.float(), score.float())
        
                predy = np.concatenate((predy, output.cpu().detach().numpy()), axis=0)
                y = np.concatenate((y, score.cpu().detach().numpy()), axis=0)

                total_loss += loss.item()

        nmse = np.sum((y - predy) ** 2) / np.sum(y ** 2)
        mse = np.sum((y - predy) ** 2) / len(train_idx)
        mae = np.sum(np.abs(y - predy)) / len(train_idx)
        meany = np.mean(y)
        rsquared = np.sum((predy - meany) ** 2) / np.sum((y - meany) ** 2)
        print('epoch {}, loss {}, nmse {}, mae {}, rsquared {}'.format(i, total_loss, nmse, mae, rsquared))
        writer.add_scalar('Loss', total_loss, i)
        writer.add_scalar('NMSE', nmse, i)
        writer.add_scalar('MAE', mae, i)
        writer.add_scalar('rsquared', rsquared, i)

writer.close()

In [ ]:
predy = []
y = []
for fold, (train_idx, val_idx) in enumerate(kf.split(np.arange(len(dataset)))):
    train_sampler = sampler.SubsetRandomSampler(train_idx)
    val_sampler = sampler.SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size=64, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=64, sampler=val_sampler)
    net.eval()
    with torch.no_grad():
        for step, [img, labels, score] in enumerate(train_loader):
            img = img.cuda()
            labels = labels.cuda()
            score = score.cuda()
            output = net(img, labels).squeeze(-1)
            loss = loss_fn(output.float(), score.float())
    
            predy = np.concatenate((predy, output.cpu().detach().numpy()), axis=0)
            y = np.concatenate((y, score.cpu().detach().numpy()), axis=0)

            total_loss += loss.item()

    nmse = np.sum((y - predy) ** 2) / np.sum(y ** 2)
    mse = np.sum((y - predy) ** 2) / len(train_idx)
    mae = np.sum(np.abs(y - predy)) / len(train_idx)
    meany = np.mean(y)
    rsquared = np.sum((predy - meany) ** 2) / np.sum((y - meany) ** 2)
    print(rsquared)
    print(mse)
    print(mae)
    break